# Проверка качества данных: Источник ЕЦП.CRM

**Таблица:** `sandbox_ai.tmp_crm_svy`  
**Формат источника:** csv  
**Содержание:** мастер-система ФП/СФП — централизованное хранение факторов проблемности  
**Объём:** ~1.5 млн строк, 41 колонка  

> Все проверки оптимизированы через векторизованные операции pandas (без iterrows).

## 0. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# === НАСТРОЙКИ ===
FILE_PATH = "../sources/data_crm.csv"
ENCODING = "utf-8-sig"
SEP = ";"

KNOWN_SOURCES = {"H2O", "CRM", "АБС", "Diasoft", "ППРБ"}
KNOWN_STATUSES_FP = {"Открыт", "Закрыт", "В работе", "На согласовании"}
KNOWN_TYPES_FP = {"ФП", "СФП"}
KNOWN_CURRENCIES = {"RUB", "USD", "EUR", "CNY", "GBP", "CHF"}
KNOWN_YN = {"Y", "N"}

DATE_FORMAT = "%d.%m.%Y"
DATE_MIN = datetime(2018, 1, 1)
DATE_MAX = datetime(2026, 12, 31)

DATE_COLUMNS = [
    "DATE_END_FP_SFP", "END_DATE_SCR_FCT", "END_DATE_SCR_PLAN",
    "END_EVENT_DATE_FACT", "FIRST_END_DATE_EVENT", "IDENTIFICATION_DATE",
    "NEW_PLAN_END_DATE_EVT", "AGREEMENT_OPEN_DT", "AGREEMENT_CLOSE_DT",
]
REQUIRED_FIELDS = ["ROW_ID", "X_INN", "IDENTIFICATION_DATE", "TYPE_FP", "VAL_1"]

In [ ]:
# Загрузка CSV — все как строки, low_memory=False для стабильности на 1.5 млн строк
df = pd.read_csv(FILE_PATH, sep=SEP, encoding=ENCODING, dtype=str, low_memory=False)
df.columns = df.columns.str.strip()

print(f"Загружено строк: {len(df):,}, колонок: {len(df.columns)}")
print(f"Колонки: {list(df.columns)}")
df.head(3)

In [ ]:
# Вспомогательная функция — проверка на пустоту (векторизованная)
def is_empty(s):
    return s.isna() | (s.astype(str).str.strip() == "")

## 1. Пропуски (NULL / пустые значения)

In [ ]:
nulls = df.apply(lambda c: is_empty(c).sum())
pct = (nulls / len(df) * 100).round(1)
null_report = pd.DataFrame({"Пропусков": nulls, "%": pct})
null_report["Статус"] = null_report["Пропусков"].apply(lambda x: "[!]" if x > 0 else "OK")
display(null_report)

for col in REQUIRED_FIELDS:
    if col in df.columns and nulls.get(col, 0) > 0:
        print(f"\n!!! КРИТИЧНО: '{col}' — {nulls[col]:,} пропусков!")

## 2. Дубликаты

In [ ]:
print(f"Полные дубликаты строк: {df.duplicated().sum():,}")

# ROW_ID — уникальный ID ФП
if "ROW_ID" in df.columns:
    clean = df[~is_empty(df["ROW_ID"])]
    dupes = clean["ROW_ID"].duplicated().sum()
    print(f"Дубликаты ROW_ID: {dupes:,}")
    if dupes > 0:
        examples = clean[clean["ROW_ID"].duplicated(keep=False)]["ROW_ID"].unique()[:5]
        print(f"  Примеры: {examples.tolist()}")

# ИНН — повторы ожидаемы
if "X_INN" in df.columns:
    inn_counts = df["X_INN"].value_counts()
    print(f"\nУникальных ИНН: {df['X_INN'].nunique():,}")
    print(f"ИНН с >1 записью: {(inn_counts > 1).sum():,}")
    print(f"\nТоп-5 ИНН по количеству записей:")
    display(inn_counts.head(5).rename_axis("ИНН").reset_index(name="Записей"))

## 3. Формат и валидность значений

In [ ]:
# --- ИНН, КПП, ОГРН ---
for col, label, lengths in [("X_INN", "ИНН", [10, 12]), ("X_KPP", "КПП", [9]), ("X_OGRN", "ОГРН", [13, 15])]:
    if col not in df.columns:
        continue
    c = df[col].dropna().astype(str).str.strip()
    c = c[c != ""]
    bad_chars = (~c.str.match(r'^\d+$')).sum()
    bad_len = (~c.str.len().isin(lengths)).sum()
    print(f"{label} ({col}):")
    print(f"  Не только цифры: {bad_chars:,}")
    print(f"  Недопустимая длина: {bad_len:,}")
    print(f"  Распределение по длине: {dict(c.str.len().value_counts().sort_index())}")
    print()

In [ ]:
# --- Все даты (9 полей) ---
print(f"Проверка дат (формат, диапазон {DATE_MIN.year}–{DATE_MAX.year}):")
date_results = []
for dcol in DATE_COLUMNS:
    if dcol not in df.columns:
        continue
    raw = df[dcol].astype(str).str.strip()
    filled = raw[(raw != "") & (raw != "nan") & (raw != "None")]
    if len(filled) == 0:
        date_results.append({"Поле": dcol, "Заполнено": 0, "Некорр. формат": 0, f"Ранее {DATE_MIN.year}": 0, f"Позднее {DATE_MAX.year}": 0})
        continue
    parsed = pd.to_datetime(filled, format=DATE_FORMAT, errors="coerce")
    valid = parsed.dropna()
    date_results.append({
        "Поле": dcol,
        "Заполнено": len(filled),
        "Некорр. формат": parsed.isna().sum(),
        f"Ранее {DATE_MIN.year}": (valid < DATE_MIN).sum(),
        f"Позднее {DATE_MAX.year}": (valid > DATE_MAX).sum(),
    })
display(pd.DataFrame(date_results))

In [ ]:
# --- APPROVED_SUM ---
if "APPROVED_SUM" in df.columns:
    sums = pd.to_numeric(df["APPROVED_SUM"], errors="coerce")
    non_num = sums.isna().sum() - df["APPROVED_SUM"].isna().sum()
    print(f"APPROVED_SUM:")
    print(f"  Нечисловые: {non_num:,}")
    print(f"  Отрицательные: {(sums < 0).sum():,}")
    print(f"  Нулевые: {(sums == 0).sum():,}")
    valid_sums = sums.dropna()
    if len(valid_sums) > 0:
        print(f"  Диапазон: {valid_sums.min():,.2f} — {valid_sums.max():,.2f}")

# --- Флаги Y/N ---
for col in ["DEFOLT", "AGR_OPEN_DT_FLG"]:
    if col not in df.columns:
        continue
    vals = set(df[col].dropna().astype(str).str.strip().unique()) - {""}
    unexpected = vals - KNOWN_YN
    status = "[OK]" if not unexpected else f"[!] Неожиданные: {sorted(unexpected)}"
    print(f"\n{col}: {sorted(vals)} {status}")

## 4. Справочные значения (допустимые перечни)

In [ ]:
ref_checks = [
    ("VAL", "Система-источник", KNOWN_SOURCES),
    ("VAL_1", "Статус ФП/СФП", KNOWN_STATUSES_FP),
    ("TYPE_FP", "Тип (ФП/СФП)", KNOWN_TYPES_FP),
    ("CURCY_CD", "Валюта", KNOWN_CURRENCIES),
]

for col, label, known in ref_checks:
    if col not in df.columns:
        continue
    vals = set(df[col].dropna().astype(str).str.strip().unique()) - {""}
    unknown = vals - known
    print(f"{col} ({label}): {sorted(vals)}")
    if unknown:
        print(f"  [?] Неизвестные: {sorted(unknown)}")
    else:
        print(f"  [OK]")
    display(df[col].value_counts().head(10).rename_axis("Значение").reset_index(name="Кол-во"))
    print()

In [ ]:
# Статусы компании и стадии сделки — для ручного анализа
for col, label in [("STATUS", "Статус (СПАРК)"), ("VAL_3", "Статус (внутр.)"), ("VAL_2", "Стадия сделки")]:
    if col not in df.columns:
        continue
    print(f"{col} ({label}):")
    display(df[col].value_counts().rename_axis("Значение").reset_index(name="Кол-во"))
    print()

## 5. Логическая согласованность

In [ ]:
def pdate(series):
    return pd.to_datetime(series, format=DATE_FORMAT, errors="coerce")

results = []

# Статус Закрыт -> DATE_END_FP_SFP заполнена
if "VAL_1" in df.columns and "DATE_END_FP_SFP" in df.columns:
    closed = df[df["VAL_1"].astype(str).str.strip() == "Закрыт"]
    cnt = is_empty(closed["DATE_END_FP_SFP"]).sum()
    results.append(("Закрыт, но DATE_END_FP_SFP пуста", cnt))

# Статус Открыт -> DATE_END_FP_SFP пуста
if "VAL_1" in df.columns and "DATE_END_FP_SFP" in df.columns:
    opened = df[df["VAL_1"].astype(str).str.strip() == "Открыт"]
    cnt = (~is_empty(opened["DATE_END_FP_SFP"])).sum()
    results.append(("Открыт, но DATE_END_FP_SFP заполнена", cnt))

# DATE_END_FP_SFP >= IDENTIFICATION_DATE
if "IDENTIFICATION_DATE" in df.columns and "DATE_END_FP_SFP" in df.columns:
    id_dt, end_dt = pdate(df["IDENTIFICATION_DATE"]), pdate(df["DATE_END_FP_SFP"])
    cnt = (id_dt.notna() & end_dt.notna() & (end_dt < id_dt)).sum()
    results.append(("DATE_END_FP_SFP < IDENTIFICATION_DATE", cnt))

# AGREEMENT_CLOSE_DT >= AGREEMENT_OPEN_DT
if "AGREEMENT_OPEN_DT" in df.columns and "AGREEMENT_CLOSE_DT" in df.columns:
    o, c = pdate(df["AGREEMENT_OPEN_DT"]), pdate(df["AGREEMENT_CLOSE_DT"])
    cnt = (o.notna() & c.notna() & (c < o)).sum()
    results.append(("AGREEMENT_CLOSE_DT < AGREEMENT_OPEN_DT", cnt))

# END_DATE_SCR_PLAN >= IDENTIFICATION_DATE
if "END_DATE_SCR_PLAN" in df.columns and "IDENTIFICATION_DATE" in df.columns:
    p, i = pdate(df["END_DATE_SCR_PLAN"]), pdate(df["IDENTIFICATION_DATE"])
    cnt = (p.notna() & i.notna() & (p < i)).sum()
    results.append(("END_DATE_SCR_PLAN < IDENTIFICATION_DATE", cnt))

# APPROVED=Y и DISABLED=Y
if "APPROVED" in df.columns and "DISABLED" in df.columns:
    cnt = ((df["APPROVED"].astype(str).str.strip() == "Y") & (df["DISABLED"].astype(str).str.strip() == "Y")).sum()
    results.append(("APPROVED=Y и DISABLED=Y одновременно", cnt))

display(pd.DataFrame(results, columns=["Проверка", "Нарушений"]))

In [ ]:
# --- DEFOLT=Y при STATUS=Действующая ---
if "DEFOLT" in df.columns and "STATUS" in df.columns:
    cnt = ((df["DEFOLT"].astype(str).str.strip() == "Y") & (df["STATUS"].astype(str).str.strip() == "Действующая")).sum()
    print(f"DEFOLT=Y при STATUS='Действующая': {cnt:,}")
    print("  (Не ошибка, но стоит учесть)")

# --- Расхождение STATUS (СПАРК) и VAL_3 (внутр.) ---
if "STATUS" in df.columns and "VAL_3" in df.columns:
    st = df["STATUS"].astype(str).str.strip()
    v3 = df["VAL_3"].astype(str).str.strip()
    m1 = ((st == "Действующая") & (v3 == "Не действующая")).sum()
    m2 = ((st == "Не действующая") & (v3 == "Действующая")).sum()
    print(f"\nРасхождение STATUS и VAL_3: {m1 + m2:,}")
    print(f"  СПАРК=Действующая, Внутр=Не действующая: {m1:,}")
    print(f"  СПАРК=Не действующая, Внутр=Действующая: {m2:,}")

## 6. Статистический профиль

In [ ]:
# По типу, статусу, источнику
for col, label in [("TYPE_FP", "Тип ФП/СФП"), ("VAL_1", "Статус"), ("VAL", "Источник"), ("DEFOLT", "Дефолт")]:
    if col in df.columns:
        print(f"{label}:")
        display(df[col].value_counts().rename_axis("Значение").reset_index(name="Кол-во"))
        print()

In [ ]:
# По годам выявления
if "IDENTIFICATION_DATE" in df.columns:
    dates = pd.to_datetime(df["IDENTIFICATION_DATE"], format=DATE_FORMAT, errors="coerce")
    print("По годам выявления:")
    display(dates.dt.year.value_counts().sort_index().rename_axis("Год").reset_index(name="Кол-во"))

# Суммы
if "APPROVED_SUM" in df.columns:
    sums = pd.to_numeric(df["APPROVED_SUM"], errors="coerce").dropna()
    print(f"\nAPPROVED_SUM: среднее={sums.mean():,.0f}, медиана={sums.median():,.0f}, мин={sums.min():,.0f}, макс={sums.max():,.0f}")

# Продукты
if "NAME" in df.columns:
    print(f"\nТоп-10 продуктов:")
    display(df["NAME"].value_counts().head(10).rename_axis("Продукт").reset_index(name="Кол-во"))

# Сценарии
if "SCRIPT" in df.columns:
    print(f"\nСценарии:")
    display(df["SCRIPT"].fillna("(пусто)").replace("", "(пусто)").value_counts()
            .rename_axis("Сценарий").reset_index(name="Кол-во"))